### Shared Word

In [7]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [8]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [9]:
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language
word_start = 0  # 0
word_end = 28  # 28 

In [10]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [11]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [12]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
1005970,karnaya,5
1005971,dörtlümüzün,5
1005972,karnavalınız,5
1005973,hurmanın,5


In [13]:
df_word = df_word_all.iloc[word_start:word_end,]
df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,o,5013838
7,ben,4908913
8,de,4880315
9,çok,4852169


In [8]:
#lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
#lang_pair_list

In [14]:
#df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{lang_pair.lower().capitalize()}_Shared_Vocabulary.xlsx")
#df_pair = df_pair.head()
df_pair

,dict_entry_main,english_word
0,abaküs,abacus
1,abandone,abandon
2,abanoz,ebony
3,abdest,ablution
4,abdesthane,ablution room
...,...,...
2132,zodyak,zodiac
2133,zombi,zombie
2134,zooloji,zoology
2135,zoolojik,zoological


In [15]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [16]:
word_select = df_word["word"].values.tolist()

In [17]:
words = df_pair["dict_entry_main"].values.tolist()

In [18]:
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [19]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [20]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)    

#### Concat Result

In [21]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,salam
1,depresyon
2,meteoroloji
3,arbitraj
4,poligon
...,...
2152,akrobasi
2153,enstitü
2154,polemik
2155,vampir


In [22]:
# Option For Frequency
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
1005970,karnaya,5
1005971,dörtlümüzün,5
1005972,karnavalınız,5
1005973,hurmanın,5


In [23]:
# Option
df_word = pd.merge(df_word,df_word_all, how="inner", on="word")
df_word.drop_duplicates(inplace=True)
df_word.reset_index(drop=True, inplace=True)
df_word

,word,frequency
0,salam,1768
1,depresyon,3215
2,meteoroloji,1212
3,arbitraj,39
4,poligon,158
...,...,...
2194,akrobasi,312
2195,enstitü,626
2196,polemik,41
2197,vampir,23754


In [39]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,var mı,281902
4,ne kadar,237876
...,...,...
17627,seni patates,6
17628,as kadar,6
17629,bu pano,6
17630,as daha,6


In [40]:
# Option For Twogram Sentence
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent = df_twogram_sent.loc[:,["twogram"]]
df_twogram_sent

,twogram
0,teşekkür ederim
1,öyle mi
2,ne oldu
3,aman tanrım
4,özür dilerim
...,...
1036515,güzeldi tommy
1036516,durumu tuhaflaştırma
1036517,güzeldi canım
1036518,güzeldi daniel


In [41]:
# Option
df_shared_twogram = pd.merge(df_shared_twogram, df_twogram_sent, how="inner", on="twogram")
df_shared_twogram.drop_duplicates(inplace=True)
df_shared_twogram.reset_index(drop=True, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,var mı,281902
4,ne kadar,237876
...,...,...
3102,bu star,6
3103,o aktör,6
3104,ve krep,6
3105,asansör de,6


In [28]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,bir şey değil,68498
4,bir şey mi,64933
...,...,...
13141,şeker mi var,10
13142,bir dj var,10
13143,kantin ve kulüp,10
13144,şeker için bana,10


In [29]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,bir şey var mı,41773
1,bir şey daha var,12185
2,bir şey mi var,11834
3,ne var ne yok,9276
4,bir şey değil mi,5080
...,...,...
5187,ben bir profesyonel değil,7
5188,kredi var ve ben,7
5189,bir bebek gibi bir,7
5190,ben amerikalı bir polis,7


In [30]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,komik bir şey mi var,901
1,bir şey yok değil mi,893
2,o kadar çok şey var,790
3,bir şey yok bir şey,490
4,ama bir şey daha var,393
...,...,...
601,ben ve bu da sen,10
602,ama bu da normal değil,10
603,ama bu beni daha da,10
604,bir şey yok ve bunu,10


In [31]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,ne var,62532
1,ben de,59972
2,değil mi,58386
3,bu da ne,50098
4,ben mi,33652
...,...,...
6423,o o kadar,6
6424,bir as,6
6425,ve abrakadabra,6
6426,evet restoran,6


In [42]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,salam,1768.0,bir şey,859944.0,bir şey yok,113165,bir şey var mı,41773.0,komik bir şey mi var,901.0,ne var,62532.0
1,depresyon,3215.0,değil mi,585879.0,bir şey var,110455,bir şey daha var,12185.0,bir şey yok değil mi,893.0,ben de,59972.0
2,meteoroloji,1212.0,ben de,377765.0,bu da ne,89463,bir şey mi var,11834.0,o kadar çok şey var,790.0,değil mi,58386.0
3,arbitraj,39.0,var mı,281902.0,bir şey değil,68498,ne var ne yok,9276.0,bir şey yok bir şey,490.0,bu da ne,50098.0
4,poligon,158.0,ne kadar,237876.0,bir şey mi,64933,bir şey değil mi,5080.0,ama bir şey daha var,393.0,ben mi,33652.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13141,NaN,NaN,NaN,NaN,şeker mi var,10,NaN,NaN,NaN,NaN,NaN,NaN
13142,NaN,NaN,NaN,NaN,bir dj var,10,NaN,NaN,NaN,NaN,NaN,NaN
13143,NaN,NaN,NaN,NaN,kantin ve kulüp,10,NaN,NaN,NaN,NaN,NaN,NaN
13144,NaN,NaN,NaN,NaN,şeker için bana,10,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx", index=False)

##### Concat Result With Comma

In [44]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            #word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(10)  # Option
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [45]:
df_word_order_twogram = word_in_twogram(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_twogram(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_twogram(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_twogram(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_twogram(df_shared_all, "word", "sentence") 

In [46]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [47]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [48]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,absürt,çok absürt,"bu çok absürt, kadar absürt bir, çok absürt ve",bu kadar absürt bir,NaN,"bu çok absürt, bu absürt"
1,adrenalin,bu adrenalin,"bir doz adrenalin, ampul daha adrenalin, adren...",bir şey adrenalin gibi,NaN,"bu adrenalin, o adrenalin mi, adrenalin değil"
2,adres,"bir adres, adres var, ve adres, bu adres, adre...","bir adres var, adres var mı, bana bir adres, b...","bir adres var mı, yok ama bir adres, ama bir a...","yok ama bir adres var, ne telefon ne de adres","adres ne, adres var mı, adres mi, adres bu, ad..."
3,afrodizyak,"bir afrodizyak, afrodizyak mı",için bir afrodizyak,NaN,NaN,bir afrodizyak
4,ahtapot,"bir ahtapot, bu ahtapot","bir ahtapot gibi, bu bir ahtapot",sen ahtapot değil bir,NaN,"bir ahtapot, evet bu ahtapot, hayır ahtapot"
...,...,...,...,...,...,...
1313,varyant,NaN,NaN,NaN,NaN,polis yok varyant yok
1314,varyete,NaN,NaN,NaN,NaN,evet varyete
1315,vegan,NaN,NaN,NaN,NaN,vegan mı
1316,volt,NaN,NaN,NaN,NaN,bir de volt


In [49]:
# Option For Word Frequency
df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
df_word_order_join_all.drop_duplicates(inplace=True)
df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
df_word_order_join_all.reset_index(drop=True, inplace=True)
df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,bir,18835735,"bir şey, bu bir, bir daha, ben bir, bana bir, ...","bir şey yok, bir şey var, bir şey değil, bir ş...","bir şey var mı, bir şey daha var, bir şey mi v...","komik bir şey mi var, bir şey yok değil mi, bi...","bir şey değil, yok bir şey, bir şey yok, bir ş..."
1,bu,11062659,"bu kadar, bu da, bu çok, bu bir, bu ne, ama bu...","bu da ne, bu kadar mı, ve bu da, bu kadar çok,...","bu şey de ne, bu o kadar da, bir şey değil bu,...","bu da bir şey mi, bu daha bir şey değil, bu no...","bu da ne, bu ne, bu o, bu kadar, bu kadar mı, ..."
2,ne,8025880,"ne kadar, ne var, da ne, bu ne, sen ne, ne içi...","bu da ne, ne kadar da, o da ne, daha ne kadar,...","ne var ne yok, bu şey de ne, daha ne kadar var...","ne kadar çok o kadar, ne sen ne de ben, ne kad...","ne var, bu da ne, ne için, bu ne, ne kadar, ne..."
3,ve,7766036,"ve ben, ve bu, ve bir, sen ve, ve sen, ve o, v...","sen ve ben, ve ben de, ve sen de, ve bu da, ve...","ve bir şey daha, ve sen de bunu, ve ben de onu...","ve bir şey daha var, ve bir o kadar da, ve bir...","sen ve ben, ve sen, ve ben, ve bu, ve ne, ve b..."
4,için,5484109,"ne için, için mi, şey için, için değil, kahve ...","için bir şey, için ne kadar, bir şey için, içi...","için bir şey var, için çok ama çok, için bana ...","için bir şey var mı, için bir şey mi var, için...","ne için, bu ne için, o ne için, şey için, ne i..."
...,...,...,...,...,...,...,...
1457,var,5,"var mı, ne var, şey var, daha var, evet var, v...","bir şey var, şey var mı, var değil mi, şey dah...","bir şey var mı, bir şey daha var, bir şey mi v...","komik bir şey mi var, o kadar çok şey var, ama...","ne var, evet var, bir şey daha var, ne var ne ..."
1458,vay,5,"vay bu, evet vay, vay ne, vay o, vay süper, va...","vay bu çok, vay vay bu, vay ne kadar, vay bu d...","vay vay vay bu, vay vay vay de, vay bu seni ço...","vay vay vay vay de, vay bu seni çok popüler, v...","vay süper, vay bu, vay bu da ne, vay ne, vay ç..."
1459,tsunami,5,NaN,"bir tsunami var, bir tsunami var",NaN,NaN,"tsunami mi, tsunami gibi, bu tsunami, tsunami ..."
1460,de,5,"ben de, sen de, bir de, ne de, de mi, beni de,...","ben de seni, ve ben de, evet ben de, ve sen de...","bu şey de ne, ben de bir şey, de bu değil mi, ...","ne sen ne de ben, ve bir de bu var, bunu sen d...","ben de, sen de, evet ben de, ben de seni, seni..."


In [50]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency.xlsx", index=False)

#### Copy Move And Delete

In [52]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*.xlsx")
output_file

['Turkish_English_Shared_Twogram.xlsx',
 'Turkish_English_Shared_Threegram.xlsx',
 'Turkish_English_Shared_Fourgram.xlsx',
 'Turkish_English_Shared_Fivegram.xlsx',
 'Turkish_English_Shared_Sentence.xlsx',
 'Turkish_English_Shared_Result_With_Frequency.xlsx',
 'Turkish_English_Shared_Join_Result_Without_Frequency.xlsx']

In [53]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [54]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass